In [1]:
from brat2conll import Brat2Conll, split_train_test
import glob
import os 
import shutil
import pandas as pd
import random 
import json

Recopie les fichiers text manquants dans le dossier annotated_data à partir du dossier raw_text:

In [2]:
ann_list = glob.glob('annotated_data/*.ann')
txt_test = [x.replace('.ann','.txt') for x in ann_list]
for txt_target in txt_test: 
    if not os.path.isfile(txt_target):
        txt_source = txt_target.replace('annotated_data', 'raw_text')
        print(txt_source)
        shutil.copyfile(txt_source, txt_target)

Converti les fichiers .ann en dataframe aggrégé par phrase:

In [3]:
converter = Brat2Conll()

df = pd.concat(converter.convert("annotated_data"))
df_agg = df.groupby(['doc_id','sent_id'])\
            .agg({'token':lambda x: [tok.text for tok in list(x)],
                  'label': lambda x : list(x)})\
            .reset_index()
df_agg

,doc_id,sent_id,token,label
0,ped_008,0,"[Histoire, clinique, L’, interrogatoire, est, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,ped_008,1,"[Nous, attribuerons, par, la, suite, cette, at...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,ped_008,2,"[L’, examen, clinique, initial, trouve, un, ét...","[O, O, O, O, O, O, disorder, disorder, disorde..."
3,ped_008,3,"[L’, examen, abdominal, met, en, évidence, une...","[O, procedure, procedure, O, O, O, O, disorder..."
4,ped_008,4,"[Le, toucher, vaginal, est, sans, anomalie, no...","[O, disorder, disorder, disorder, disorder, di..."
...,...,...,...,...
24313,unil_007,26,"[Tonus, normal, .]","[O, O, O]"
24314,unil_007,27,"[ROT, faibles, aux, 4, membres, ,, avec, RCP, ...","[O, O, O, O, O, O, O, O, O, O, O]"
24315,unil_007,28,"[Force, et, sensibilité, tacto, -, algique, co...","[O, O, O, O, O, O, O, O]"
24316,unil_007,29,"[Pas, de, troubles, grossiers, de, la, coordin...","[O, O, O, O, O, O, O, O, O, O, O, O]"


Supprime les phrases avec des labels "irrelevant":

In [4]:
df_agg_filtered = df_agg[df_agg['label'].apply(lambda x: 'irrelevant' not in x)]

Transforme le dataframe en list de dictionnaires compatible avec le format json et effectue le split en train/test:

In [5]:
trainset, testset = split_train_test(df_agg_filtered, seed = 23, train_frac = 0.8)

Enregistre les train et test sets dans des fichiers json:

In [6]:
with open('datasets/trainset.json', 'w', encoding='utf8') as f: 
    json.dump(trainset, f)
with open('datasets/testset.json', 'w', encoding='utf8') as f: 
    json.dump(testset, f)